In [1]:
import pandas as pd
df = pd.read_pickle('business_data')
df= df.sample(frac=1)#Shuffling the dataset
print('file_read')

file_read


In [2]:
#Preparing the input in numpy array- step 1
import numpy as np
X_mod = df['words_vector'].tolist()
y_mod=  np.array(df['stars'].tolist())
del df
print('done')

done


In [3]:
#Preparing the input in numpy array- step 2
import numpy as np
import sys
max_words=30# This number can be increased but I am limited by my machine's memory
vector_size=100
X = np.empty((len(X_mod),max_words,vector_size))
for index1 in range(0,len(X_mod)):
    if len(X_mod[index1])< max_words:
        for index2 in range(0,max_words - len(X_mod[index1])):
             for index3 in range(0,vector_size):
                    X[index1][index2][index3]=0
        for index2 in range(0,len(X_mod[index1])):
             for index3 in range(0,vector_size):
                    X[index1][max_words - len(X_mod[index1]) + index2][index3]=X_mod[index1][index2][index3]
    else:
        for index2 in range(0,max_words):
             for index3 in range(0,vector_size):
                    X[index1][index2][index3]=X_mod[index1][index2][index3]
print('done')

done


In [5]:
#Preparing the output vector
y = np.zeros((len(y_mod),5))
for item in range(0,len(y_mod)):
    y[item][y_mod[item]-1] = 1
print('done')

done


In [6]:
#Freeing memory
del X_mod
del y_mod
np.save('X',X)
np.save('y',y)
print('done')

done


In [ ]:
#Building LSTM model for rating generation
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import LSTM
vector_embedding = 100#vector embedding of the word which is of size 100
hidden_layer =300
fixlen=30#Do not accept sentences which have more than 30 words. Truncate them
# Model
X=np.load('X.npy')
y=np.load('y.npy')
print('Building model...')
model = Sequential()
model.add(LSTM(input_dim=vector_embedding, output_dim=hidden_layer, \
    input_length=fixlen, return_sequences=True))
model.add(Activation('sigmoid'))
model.add(Dropout(0.2))
model.add(LSTM(input_dim=hidden_layer, output_dim=hidden_layer, \
               input_length=fixlen, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Train and save         

model.fit(X[0:100000], y[0:100000], batch_size=128, nb_epoch=10)
model.save_weights('model_weight.hdf5',overwrite=True)
          


Building model...


In [ ]:
# Load model and Predict
predicted_probabilties = model.predict(X[100000:], verbose=0) 
print('done')

In [34]:
#Visualise the results
import pandas as pd
real = np.argmax(y[100000:], axis =1)+1
predicted = np.argmax(predicted_probabilties, axis =1)+1
df = pd.DataFrame(real, columns =['real'])
df['predicted'] = predicted
df[df['real'] == df['predicted']]

,real,predicted
0,4,4
4,5,5
12,1,1
14,5,5
16,5,5
17,5,5
20,5,5
21,5,5
22,1,1
26,5,5


In [49]:
#AUC score for multinomial classification
from sklearn.metrics import roc_auc_score
df['correct_prediction'] = np.where(df['real'] == df['predicted'], 1, 0)
df['score'] = np.amax(predicted_probabilties, axis=1)
roc_auc_score(df['correct_prediction'], df['score'])

In [55]:
#AUC

0.71243653971605858
